In [3]:
import cv2
import numpy as np
import math
from tensorflow.keras.models import load_model
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# ===============================
# Load trained gesture model
# ===============================
model = load_model("handGesture_model.keras")

# Gesture labels (update if needed, must match training dataset)
gesture_labels = [
    "Palm", "Fist", "Thumbs Up", "Thumbs Down", "Index", 
    "Ok", "Peace", "Stop", "Call Me", "Other"
]

# ===============================
# Initialize system volume
# ===============================
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
min_vol, max_vol, _ = volume.GetVolumeRange()

# ===============================
# Webcam Setup
# ===============================
img_size = 64
cam = cv2.VideoCapture(0)
cam.set(3, 640)
cam.set(4, 480)

while cam.isOpened():
    ret, frame = cam.read()
    if not ret:
        break

    # Preprocess for model
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (img_size, img_size))
    normalized = resized.astype("float32") / 255.0
    reshaped = normalized.reshape(1, img_size, img_size, 1)

    # Predict gesture
    preds = model.predict(reshaped, verbose=0)
    class_id = np.argmax(preds)
    confidence = preds[0][class_id]
    gesture = gesture_labels[class_id]

    # ===============================
    # Map gestures to volume control
    # ===============================
    if gesture == "Thumbs Up":
        current_vol = volume.GetMasterVolumeLevelScalar()
        new_vol = min(current_vol + 0.05, 1.0)  # Increase volume
        volume.SetMasterVolumeLevelScalar(new_vol, None)
    elif gesture == "Thumbs Down":
        current_vol = volume.GetMasterVolumeLevelScalar()
        new_vol = max(current_vol - 0.05, 0.0)  # Decrease volume
        volume.SetMasterVolumeLevelScalar(new_vol, None)
    elif gesture == "Palm":
        volume.SetMute(0, None)  # Unmute
    elif gesture == "Fist":
        volume.SetMute(1, None)  # Mute

    # ===============================
    # Draw UI
    # ===============================
    vol_percent = int(volume.GetMasterVolumeLevelScalar() * 100)
    cv2.putText(frame, f"Gesture: {gesture} ({confidence*100:.1f}%)",
                (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Volume: {vol_percent}%", (10, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.rectangle(frame, (50, 150), (85, 400), (0, 255, 0), 3)
    cv2.rectangle(frame, (50, int(400 - (vol_percent / 100) * 250)), 
                  (85, 400), (0, 255, 0), cv2.FILLED)

    cv2.imshow("Hand Gesture Volume Control", frame)

    # Quit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ===============================
# Cleanup
# ===============================
cam.release()
cv2.destroyAllWindows()